In [5]:
# Cell 1: Import Libraries and Configure Notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
from datetime import datetime
from scipy.stats import spearmanr, ttest_ind, f_oneway

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

# Set seaborn style for plots
sns.set(style='whitegrid', context='notebook', palette='deep')
%matplotlib inline

In [ ]:
# Cell 2: Data Loading and JSON Parsing Functions

def parse_json_column(val):
    """Try to parse a JSON string; if it fails, return the original value."""
    try:
        # Some fields may be empty strings; treat them as NaN
        if pd.isna(val) or (isinstance(val, str) and val.strip()==''):
            return np.nan
        return json.loads(val)
    except Exception:
        return val

def load_and_process_data(file_path):
    """
    Load the artifact CSV data, convert date fields,
    and parse columns that are stored as JSON strings.
    """
    # Read the CSV file
    df = pd.read_csv(file_path, low_memory=False)
    
    # Convert date/time columns (if present)
    for col in ['_created', '_updated', 'merged_at']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Identify columns that hold JSON strings
    json_cols = ['_links', 'failed_job', 'passed_job', 'metrics', 'reproducibility_status']
    for col in json_cols:
        if col in df.columns:
            df[col] = df[col].apply(parse_json_column)
    
    # Convert numeric fields (if any appear as numbers in string form)
    numeric_cols = ['pr_num', 'reproduce_attempts', 'reproduce_successes']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

# Change file_path as needed
file_path = '/Users/harshil/Developer/GitHub_Repos/FailFix/Data/processed/artifact_data_table_44.csv'
df = load_and_process_data(file_path)
print("Data loaded. Shape:", df.shape)
df.head()

Data loaded. Shape: (3324, 33)


,_created,_deleted,_etag,_id,_links,_updated,added_version,base_branch,branch,build_system,cached,ci_service,current_image_tag,deprecated_version,failed_job,filtered_reason,image_tag,is_error_pass,lang,match,merged_at,metrics,passed_job,pr_num,repo,repo_mined_version,reproduce_attempts,reproduce_successes,reproduced,reproducibility_status,stability,status,test_framework
0,2018-08-24 08:01:18+00:00,False,f0ba3f7059f911957b0c99ba4b82c23455bbc734,5b7fbb4e37be5b494c9f9f0b,"{'collection': {'href': 'artifacts', 'title': 'artifacts'}, 'next': {'href': 'artifacts/wbond-package_control_channel-379589051?page=2', 'title': 'next page'}, 'parent': {'href': '/', 'title': 'home'}, 'self': {'href': 'artifacts/wbond-package_control_channel-379589051', 'title': 'Artifact'}}",2024-07-02 16:30:43+00:00,1.0.0,master,master,NaN,True,travis,wbond-package_control_channel-379589051,NaN,"{'base_sha': '93f88173b6f55ce9a46c2e56dc1eeb8d0da2243b', 'build_id': 379589050, 'build_job': '14497.1', 'committed_at': '2018-05-16T07:40:38Z', 'component_versions': {'analyzer': '96ba284ceef7e8732a2cfd6b769b337c4479f487', 'reproducer': 'ebdf7a233e8d2d850acf9c5c8ab031f11421eef5'}, 'config': {'': {'result': 'configured'}, 'dist': 'trusty', 'group': 'stable', 'language': 'python', 'os': 'linux', 'python': '3.3', 'script': ['python -m unittest', 'curl -X POST https://packagecontrol.io/test_pr/$TRAVIS_PULL_REQUEST.json'], 'sudo': False}, 'failed_tests': 'test_repository_package_names('./repository/c.json', ...) (tests.test.DefaultRepositoryTests)', 'is_git_repo': True, 'job_id': 379589051, 'message': 'retitle Chrome REPL > ChromeREPL to fix imports', 'mismatch_attrs': [], 'num_tests_failed': 1, 'num_tests_run': 8451, 'patches': {}, 'trigger_sha': '9a4cd38cf59908e1be6f75ce695ccc0a146250aa'}",NaN,wbond-package_control_channel-379589051,False,Python,NaN,2018-05-17 13:11:19+00:00,"{'additions': 43, 'changes': 52, 'deletions': 9, 'num_of_changed_files': 5}","{'base_sha': 'acbd921114ac46d62e86fd2cf98a06a156627d47', 'build_id': 380083244, 'build_job': '14504.1', 'committed_at': '2018-05-17T08:06:11Z', 'component_versions': {'analyzer': '96ba284ceef7e8732a2cfd6b769b337c4479f487', 'reproducer': 'ebdf7a233e8d2d850acf9c5c8ab031f11421eef5'}, 'config': {'': {'result': 'configured'}, 'dist': 'trusty', 'group': 'stable', 'language': 'python', 'os': 'linux', 'python': '3.3', 'script': ['python -m unittest', 'curl -X POST https://packagecontrol.io/test_pr/$TRAVIS_PULL_REQUEST.json'], 'sudo': False}, 'failed_tests': '', 'is_git_repo': True, 'job_id': 380083245, 'message': 'alphabetize ChromeREPL', 'mismatch_attrs': [], 'num_tests_failed': 0, 'num_tests_run': 8457, 'patches': {}, 'trigger_sha': 'f66141cda9a4149e962c5d4da5d6c8f96bd0f85c'}",7113,wbond/package_control_channel,efabcb148a97bb217fbd0c75655bd885bf6ec06d,5.0,5.0,False,"{'status': 'Reproducible', 'time_stamp': '2018-08-24'}",5/5,active,unittest
1,2018-03-06 04:13:22+00:00,False,4349909dfe62bd5dd30c5b1b24951dbdc14a4f31,5a9e1562f1f70e072f3a6672,"{'collection': {'href': 'artifacts', 'title': 'artifacts'}, 'next': {'href': 'artifacts/checkstyle-checkstyle-248927615?page=2', 'title': 'next page'}, 'parent': {'href': '/', 'title': 'home'}, 'self': {'href': 'artifacts/checkstyle-checkstyle-248927615', 'title': 'Artifact'}}",2024-07-02 16:29:29+00:00,1.0.0,NaN,master,Maven,True,travis,checkstyle-checkstyle-248927615,NaN,"{'base_sha': '', 'build_id': 248927610, 'build_job': '11011.5', 'committed_at': '2017-06-30T20:23:46Z', 'config': {'': {'result': 'configured'}, 'addons': {'apt': {'packages': ['xsltproc', 'xmlstarlet', 'oracle-java8-installer']}}, 'after_success': ['set -e if [[ -n $CMD_AFTER_SUCCESS && $SKIP_CI == 'false' ]]; then eval $CMD_AFTER_SUCCESS; echo ""CMD_AFTER_SUCCESS is finished""; fi ', 'set -e SKIP_DEPLOY=$(if [ $(git log -1 | grep -E ""\[maven-release-plugin\] prepare release"" | cat | wc -l) -lt 1 ]; then echo false; else echo true; fi;) if [[ $TRAVIS_REPO_SLUG == 'checkstyle/checkstyle' && $TRAVIS_BRANCH == 'master' && $

In [7]:
# Cell 3: Basic Data Exploration and Quality Report

def generate_data_quality_report(df):
    total_rows = len(df)
    
    # Missing value report
    missing_data = pd.DataFrame({
        'Missing Count': df.isnull().sum(),
        'Missing Percentage': (df.isnull().sum()/total_rows*100).round(2)
    })
    
    # Categorical columns to inspect
    categorical_cols = ['lang', 'build_system', 'ci_service', 'status', 'test_framework']
    value_distributions = {col: df[col].value_counts().head(10) for col in categorical_cols if col in df.columns}
    
    return missing_data, value_distributions

missing_report, cat_distributions = generate_data_quality_report(df)
print("Missing Data Report:")
print(missing_report)
print("\nValue Distributions for Key Categorical Columns:")
for col, dist in cat_distributions.items():
    print(f"\n{col}:")
    print(dist)

Missing Data Report:
                        Missing Count  Missing Percentage
_created                            0                0.00
_deleted                            0                0.00
_etag                               0                0.00
_id                                 0                0.00
_links                              0                0.00
_updated                            0                0.00
added_version                       0                0.00
base_branch                      2629               79.09
branch                              0                0.00
build_system                     1264               38.03
cached                              0                0.00
ci_service                          0                0.00
current_image_tag                   0                0.00
deprecated_version               3324              100.00
failed_job                          0                0.00
filtered_reason                  3324              